In [ ]:
# google drive 연동
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os, pandas as pd
os.chdir('/content/drive/MyDrive/jiwon')

weather = pd.read_csv('해운대 날씨.csv')
customer = pd.read_csv('해운대 입장객수2.csv')

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers import Bidirectional
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import MaxAbsScaler,RobustScaler
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
import datetime
import time
import inspect

# 날짜를 date type으로 변경 후, 나머지는 numeric type으로 변경
weather['날짜'] = pd.to_datetime(weather['날짜'], infer_datetime_format=True)
weather.iloc[:,1:] = weather.iloc[:,1:].apply(pd.to_numeric)

customer['방문일'] = pd.to_datetime(customer['방문일'], infer_datetime_format=True)
customer['방문객수'] = customer['방문객수'].str.replace(",","")
customer.iloc[:,1:] = customer.iloc[:,1:].apply(pd.to_numeric)

In [ ]:
# merge data : 데이터 합치기
total_data = pd.merge(weather, customer, left_on='날짜', right_on="방문일", how='inner')
total_data = total_data[['강수_관측값', "기온", "습도", "체감온도", "평균수온", "평균풍속", "평균기압", "평균최대파고", "평균파주기", "방문객수"]]


In [ ]:
# 결측치 평균, 0 채우기
total_data.loc[total_data['평균수온'] != total_data['평균수온'], '평균수온'] = total_data['평균수온'].mean()
total_data.loc[total_data['평균풍속'] != total_data['평균풍속'], '평균풍속'] = total_data['평균풍속'].mean()
total_data.loc[total_data['평균기압'] != total_data['평균기압'], '평균기압'] = total_data['평균기압'].mean()
total_data.loc[total_data['평균최대파고'] != total_data['평균최대파고'], '평균최대파고'] = total_data['평균최대파고'].mean()
total_data.loc[total_data['평균파주기'] != total_data['평균파주기'], '평균파주기'] = total_data['평균파주기'].mean()
total_data.loc[total_data['방문객수'] != total_data['방문객수'], '방문객수'] = total_data['방문객수'].mean()
# total_data = total_data.fillna(0)

In [ ]:
input_set = total_data[["강수_관측값", "기온", "습도", "체감온도", "평균풍속", "평균기압", "평균수온", "평균최대파고", "평균파주기", "방문객수"]]
output_set = total_data[['강수_관측값', '기온', "방문객수"]]

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(input_set, output_set, test_size=0.2)

In [ ]:
train_x.shape, test_x.shape

((1172, 10), (293, 10))

In [ ]:
scaler = MinMaxScaler()
train_x_scaled = scaler.fit_transform(train_x)
test_x_scaled = scaler.transform(test_x)

In [ ]:
train_x=train_x_scaled.reshape(1172, 5, 2)
train_x = train_y.astype(float)
test_x=test_x_scaled.reshape(293, 5, 2)
test_x = test_y.astype(float)

In [ ]:
train_y = train_y.astype(float)
test_y = test_y.astype(float)

In [ ]:
train_x.shape, test_x.shape

((1172, 3), (293, 3))

In [ ]:
# 2. 모델구성
model=Sequential()
model.add(LSTM(units=256,input_shape=(5,2),return_sequences=True))
model.add(Bidirectional(LSTM(512)))
model.add(Dense(256, activation='swish'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='swish'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='swish'))
model.add(Dense(3))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 5, 256)            265216    
                                                                 
 bidirectional (Bidirectiona  (None, 1024)             3149824   
 l)                                                              
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                        

In [ ]:
# 3. 컴파일 훈련
earlyStopping = EarlyStopping(monitor='val_loss', patience=30, mode='min', verbose=1,restore_best_weights=True)
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer)
hist = model.fit(train_x_scaled, train_y, epochs=500, batch_size=32, 
                validation_split=0.2,
                callbacks = [earlyStopping],
                verbose=2)